#비교결과
- CNN : accuracy=0.855
- RNN : accuracy=0.801
- GRU : accuracy=0.806
- LSTM : accuracy=0.826

##accuracy : CNN > LSTM > GRU > RNN
##speed : LSTM > GRU = RNN > CNN
 
정확도와 속도적인 면을 봤을때 CNN을 사용하는 것이 좋을 것 같다.
각 모델들의 epoch나 일정부분이 다른 것은 오히려 accuracy 가 떨어지는 경우가 있어서... 각각 돌렸을 때 나왔던 최고의 accuracy를 기입함.

#CNN

In [25]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
train_data[0] # [1,14,22, ... , 19,178,32]
train_labels[0] # 1
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value,key) for (key, value) in word_index.items()]
)
decode_review = ' '.join(
    [reverse_word_index.get(i-3, '?') for i in train_data[0]]
)

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
  
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16,activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs = 20,
                    batch_size = 512,
                    validation_data=(x_val, y_val))
history_dict = history.history

import matplotlib.pyplot as plt

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1,len(loss)+1)

# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# history = model.fit(partial_x_train,
#                     partial_y_train,
#                     epochs = 5,
#                     batch_size = 512,
#                     validation_data=(x_val, y_val))

#results = model.evaluate(x_test, y_test)
loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)
print('Test performance: accuracy={0}, loss={1}'.format(acc, loss))

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 2s 115us/step - loss: 0.5378 - accuracy: 0.7679 - val_loss: 0.4208 - val_accuracy: 0.8482
Epoch 2/20
15000/15000 [==============================] - 2s 107us/step - loss: 0.3308 - accuracy: 0.9003 - val_loss: 0.3163 - val_accuracy: 0.8896
Epoch 3/20
15000/15000 [==============================] - 2s 107us/step - loss: 0.2395 - accuracy: 0.9253 - val_loss: 0.3133 - val_accuracy: 0.8736
Epoch 4/20
15000/15000 [==============================] - 2s 108us/step - loss: 0.1867 - accuracy: 0.9397 - val_loss: 0.3001 - val_accuracy: 0.8841
Epoch 5/20
15000/15000 [==============================] - 2s 108us/step - loss: 0.1483 - accuracy: 0.9529 - val_loss: 0.2949 - val_accuracy: 0.8824
Epoch 6/20
15000/15000 [==============================] - 2s 109us/step - loss: 0.1157 - accuracy: 0.9669 - val_loss: 0.3450 - val_accuracy: 0.8704
Epoch 7/20
15000/15000 [==============================] - 2s 1

# RNN

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras import layers, models, datasets
from keras.preprocessing import sequence

class MyRNNCell(layers.Layer):
	def __init__(self, units, **kwargs):
		self.units = units
		self.state_size = units
		super(MyRNNCell, self).__init__(**kwargs)

	def build(self, input_shape):
		self.input_kernel = self.add_weight( shape=(input_shape[-1], self.units), initializer='uniform', name='input_kernel')
		self.hidden_kernel = self.add_weight( shape=(self.units, self.units), initializer='uniform', name='hidden_kernel')
		self.output_kernel = self.add_weight( shape=(self.units, self.units), initializer='uniform', name='output_kernel')
		self.hidden_bias = self.add_weight( shape=(self.units,), initializer='zeros', name='hidden_bias')
		self.output_bias = self.add_weight( shape=(self.units,), initializer='zeros', name='output_bias')
		self.built = True

	def call(self, inputs, states):
		prev_hidden = states[0]

		h = K.dot(inputs, self.input_kernel) + K.dot(prev_hidden, self.hidden_kernel)
		h = K.tanh(h + self.hidden_bias)
		output = K.dot(h, self.output_kernel) + self.output_bias
		return output, [h]

	def compute_output_shape(self, input_shape):
		return (input_shape[0], self.units)

max_features = 20000
maxlen = 80
batch_size = 100
epochs = 4

(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x = layers.Input((maxlen,))
h = layers.Embedding(max_features, 128)(x)
h = layers.RNN(MyRNNCell(128))(h)
h = layers.Activation('relu')(h)
h = layers.Dropout(0.25)(h)
y = layers.Dense(1, activation='sigmoid')(h)

model = models.Model(x, y)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

print('Training stage')

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=2)
# plt.plot(history.history['loss'], 'y', label='train loss')
# plt.plot(history.history['val_loss'], 'r', label='val loss')
# plt.legend(loc='upper right')
# plt.show()
# plt.plot(history.history['acc'], 'y', label='train acc')
# plt.plot(history.history['val_acc'], 'r', label='val acc')
# plt.legend(loc='upper right') 
# plt.show()

loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)
print('Test performance: accuracy={0}, loss={1}'.format(acc, loss))

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 80)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 80, 128)           2560000   
_________________________________________________________________
rnn_10 (RNN)                 (None, 128)               49408     
_________________________________________________________________
activation_10 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 129       
Total params: 2,609,537
Trainable params: 2,609,537
Non-trainable params: 0
________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
 - 32s - loss: 0.5182 - accuracy: 0.7212 - val_loss: 0.3924 - val_accuracy: 0.8275
Epoch 2/4
 - 32s - loss: 0.3105 - accuracy: 0.8730 - val_loss: 0.4217 - val_accuracy: 0.8119
Epoch 3/4
 - 32s - loss: 0.2084 - accuracy: 0.9207 - val_loss: 0.4865 - val_accuracy: 0.7833
Epoch 4/4
 - 32s - loss: 0.1678 - accuracy: 0.9368 - val_loss: 0.5940 - val_accuracy: 0.7983
Test performance: accuracy=0.7982800006866455, loss=0.5939794690608978


#GRU

In [16]:
class MyGRUCell(layers.Layer):
  def __init__(self, units, **kwargs):
    self.units = units
    self.state_size = units
    super(MyGRUCell, self).__init__(**kwargs)

    
  def build(self, input_shape):
    self.input_update_kernel = self.add_weight(
        shape=(input_shape[-1], self.units),
        initializer='uniform',
        name='input_update_kernel')
    self.hidden_update_kernel = self.add_weight(
        shape=(self.units, self.units),
        initializer='uniform',
        name='hidden_update_kernel')
    self.input_reset_kernel = self.add_weight(
        shape=(input_shape[-1], self.units),
        initializer='uniform',
        name='input_reset_kernel')
    self.hidden_reset_kernel = self.add_weight(
        shape=(self.units, self.units),
        initializer='uniform',
        name='hidden_reset_kernel')
    self.input_kernel = self.add_weight(
        shape=(input_shape[-1], self.units),
        initializer='uniform',
        name='input_kernel')
    self.hidden_kernel = self.add_weight(
        shape=(self.units, self.units),
        initializer='uniform',
        name='hidden_kernel')
    
    self.built = True

    
  def call(self, inputs, states):
    prev_hidden = states[0]
    
    z = K.sigmoid(K.dot(inputs, self.input_update_kernel)
                  + K.dot(prev_hidden, self.hidden_update_kernel))
    r = K.sigmoid(K.dot(inputs, self.input_reset_kernel)
                  + K.dot(prev_hidden, self.hidden_reset_kernel))
    h_t = K.tanh(K.dot(inputs, self.input_kernel)
                 + r * K.dot(prev_hidden, self.hidden_kernel))
    h = z * prev_hidden + (1 - z) * h_t
    
    return h, [h]
  
  
  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.units)

max_features = 20000
maxlen = 80
batch_size = 100
epochs = 4

(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x = layers.Input((maxlen,))
h = layers.Embedding(max_features, 128)(x)
h = layers.RNN(MyRNNCell(128))(h)
h = layers.Activation('relu')(h)
h = layers.Dropout(0.25)(h)
y = layers.Dense(1, activation='sigmoid')(h)
model = models.Model(x, y)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

print('Training stage')

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=2)
# plt.plot(history.history['loss'], 'y', label='train loss')
# plt.plot(history.history['val_loss'], 'r', label='val loss')
# plt.legend(loc='upper right')
# plt.show()

# plt.plot(history.history['acc'], 'y', label='train acc')
# plt.plot(history.history['val_acc'], 'r', label='val acc')
# plt.legend(loc='upper right') 
# plt.show()

loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)
print('Test performance: accuracy={0}, loss={1}'.format(acc, loss))

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 80, 128)           2560000   
_________________________________________________________________
rnn_6 (RNN)                  (None, 128)               49408     
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 129       
Total params: 2,609,537
Trainable params: 2,609,537
Non-trainable params: 0
_________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
 - 39s - loss: 0.5203 - accuracy: 0.7262 - val_loss: 0.4262 - val_accuracy: 0.8094
Epoch 2/4
 - 42s - loss: 0.3165 - accuracy: 0.8719 - val_loss: 0.4373 - val_accuracy: 0.8148
Epoch 3/4
 - 37s - loss: 0.2213 - accuracy: 0.9143 - val_loss: 0.4667 - val_accuracy: 0.8129
Epoch 4/4
 - 40s - loss: 0.1698 - accuracy: 0.9366 - val_loss: 0.5311 - val_accuracy: 0.8067
Test performance: accuracy=0.8066800236701965, loss=0.5311337777376175


#LSTM
- learning 시간 가장 김

In [12]:
import numpy as np
import keras
from keras import layers, models, datasets
from keras import backend as K
from keras.utils import np_utils
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
import os
 
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(
            num_words=max_features)
        x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
        x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
 
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        x = layers.Input((maxlen,))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)
 
        self.compile(loss='binary_crossentropy',
            optimizer='adam', metrics=['accuracy'])
 
class Machine:
    def __init__(self, max_features=20000, maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)
 
    def run(self, epochs=4, batch_size=32):
        data = self.data
        model = self.model
 
        print('Training stage')
        model.fit(data.x_train, data.y_train,
            batch_size=batch_size, epochs=epochs,
            validation_data=(data.x_test, data.y_test),
            verbose=2)
 
        loss, acc = model.evaluate(data.x_test, data.y_test,
            batch_size=batch_size, verbose=2)
 
        print('Test performance: accuracy={0}, loss={1}'.format(acc, loss))
 
def main():
    m = Machine()
    m.run()
 
if __name__ == '__main__':
    main()

Training stage


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
 - 140s - loss: 0.4543 - accuracy: 0.7878 - val_loss: 0.3932 - val_accuracy: 0.8269
Epoch 2/4
 - 144s - loss: 0.2986 - accuracy: 0.8810 - val_loss: 0.3840 - val_accuracy: 0.8348
Epoch 3/4
 - 141s - loss: 0.2197 - accuracy: 0.9142 - val_loss: 0.4369 - val_accuracy: 0.8324
Epoch 4/4
 - 141s - loss: 0.1534 - accuracy: 0.9431 - val_loss: 0.4903 - val_accuracy: 0.8268
Test performance: accuracy=0.8267999887466431, loss=0.49030957569122313
